# MLCL Assignment - 3
## Submitted by Chetan Patil

### Both the tasks have been trained over Polarity dataset http://www.cs.cornell.edu/people/pabo/movie-review-data/
## Task 1:
### Part a: Text Classification using CNNs on polarity Dataset

In [206]:
import tensorflow as tf
import numpy as np
import os, glob

import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder

In [207]:
sentence = []
labels = []
def read_train_dataset(child, sentence_list, labels_list, label, path = "./txt_sentoken/"):
    for root, dirs, files in os.walk(path+child):
        for file in files:
            if file.endswith(".txt"):
                f = open(path+child+"/"+file, "r")
                for line in f.readlines():
                    sentence_list.append(line)
                    labels_list.append(child)

In [208]:
read_train_dataset("pos", sentence, labels, "pos")
read_train_dataset("neg", sentence, labels, "neg")

In [209]:
print(len(sentence))

64720


In [210]:
vocab_dictionary = 10000
max_seq_len = 20
embedding_size = 50
batch_size = 64
epochs = 10
filter_size = 3
filters = 250
hidden_dims = 256

In [211]:
#preprocessing
token = Tokenizer(num_words=vocab_dictionary)
token.fit_on_texts(sentence)
x = token.texts_to_sequences(sentence)
x = sequence.pad_sequences(x, maxlen=max_seq_len)
y = labels
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

y = to_categorical(y)
print(x.shape)
print(y.shape)

(64720, 20)
(64720, 2)


In [212]:
#model
model = Sequential()

model.add(Embedding(input_dim=vocab_dictionary, output_dim=embedding_size, input_length=max_seq_len))

model.add(Dropout(0.2))
model.add(Conv1D(filters=filters, kernel_size=filter_size, strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=hidden_dims, activation='relu', kernel_regularizer=l2(0.2)))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation='softmax'))

In [215]:
#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 20, 50)            500000    
_________________________________________________________________
dropout_51 (Dropout)         (None, 20, 50)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 18, 250)           37750     
_________________________________________________________________
global_max_pooling1d_23 (Glo (None, 250)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 256)               64256     
_________________________________________________________________
dropout_52 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 2)                 514       
Total para

In [216]:
#model fitting
history = model.fit(x, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 51776 samples, validate on 12944 samples
Epoch 1/10
51776/51776 [==============================] - 27s 516us/step - loss: 0.4214 - acc: 0.8115 - val_loss: 1.3160 - val_acc: 0.4597
Epoch 2/10
51776/51776 [==============================] - 25s 489us/step - loss: 0.3912 - acc: 0.8263 - val_loss: 1.4108 - val_acc: 0.4302
Epoch 3/10
51776/51776 [==============================] - 26s 493us/step - loss: 0.3636 - acc: 0.8436 - val_loss: 1.7200 - val_acc: 0.4015
Epoch 4/10
51776/51776 [==============================] - 29s 561us/step - loss: 0.3360 - acc: 0.8547 - val_loss: 1.3481 - val_acc: 0.4713
Epoch 5/10
51776/51776 [==============================] - 24s 470us/step - loss: 0.3121 - acc: 0.8691 - val_loss: 1.4748 - val_acc: 0.5157
Epoch 6/10
51776/51776 [==============================] - 30s 586us/step - loss: 0.2835 - acc: 0.8827 - val_loss: 1.9116 - val_acc: 0.4316
Epoch 7/10
51776/51776 [==============================] - 28s 532us/step - loss: 0.2670 - acc: 0.8897 - val_loss: 1.

In [217]:
input_vec = ["I am good", "Very bad", "That is so awesome", "it stinks", "enjoying but distressed"]
input_vec = token.texts_to_sequences(input_vec)
input_vec = sequence.pad_sequences(input_vec, maxlen=max_seq_len)
label_prob = model.predict(input_vec)
label_pred = label_prob.argmax(axis=-1)
print(label_encoder.inverse_transform(label_pred));

['neg' 'pos' 'pos' 'neg' 'neg']


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Part c: Extending the approaches 

We can consider ther named entities as well as POS tags for words along with text in order to enrich the features. In case of sentiment analysis we can also consider the tf-idf values in the feature vector. We can also take care of punctuations and include them in text as well as feature vector neglecting the irrelevant punctuations.

## Task 2
### Part a:  Text Classification using RNN's (LSTM) on polarity dataset
Same above data is used in this task 

In [218]:
from keras.layers import LSTM

In [219]:
#preprocessing
token = Tokenizer(num_words=vocab_dictionary)
token.fit_on_texts(sentence)
x = token.texts_to_sequences(sentence)
x = sequence.pad_sequences(x, maxlen=max_seq_len)
y = labels
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

y = to_categorical(y)
print(x.shape)
print(y.shape)

(64720, 20)
(64720, 2)


In [223]:
model = Sequential()
model.add(Embedding(input_dim = vocab_dictionary, output_dim = embedding_size, input_length = max_seq_len))

model.add(LSTM(units = 100, recurrent_dropout=0.3, kernel_regularizer=l2(0.02), recurrent_regularizer=l2(0.02)))

model.add(Dropout(0.2))

model.add(Dense(2))

model.add(Activation('softmax'))

In [224]:
#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 20, 50)            500000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               60400     
_________________________________________________________________
dropout_54 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 2)                 202       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0         
Total params: 560,602
Trainable params: 560,602
Non-trainable params: 0
_________________________________________________________________
None


In [225]:
history = model.fit(x, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 51776 samples, validate on 12944 samples
Epoch 1/10
51776/51776 [==============================] - 32s 624us/step - loss: 0.7956 - acc: 0.6447 - val_loss: 0.9102 - val_acc: 0.3059
Epoch 2/10
51776/51776 [==============================] - 31s 592us/step - loss: 0.5773 - acc: 0.7102 - val_loss: 0.9832 - val_acc: 0.3276
Epoch 3/10
51776/51776 [==============================] - 32s 622us/step - loss: 0.5273 - acc: 0.7414 - val_loss: 0.9851 - val_acc: 0.3801
Epoch 4/10
51776/51776 [==============================] - 32s 618us/step - loss: 0.5038 - acc: 0.7541 - val_loss: 0.9044 - val_acc: 0.4814
Epoch 5/10
51776/51776 [==============================] - 33s 646us/step - loss: 0.4902 - acc: 0.7605 - val_loss: 1.0984 - val_acc: 0.3395
Epoch 6/10
51776/51776 [==============================] - 35s 674us/step - loss: 0.4806 - acc: 0.7646 - val_loss: 1.0077 - val_acc: 0.4411
Epoch 7/10
51776/51776 [==============================] - 35s 677us/step - loss: 0.4725 - acc: 0.7696 - val_loss: 1.

In [226]:
input_vec = ["I am good", "Very bad", "That is so awesome", "it stinks", "enjoying but distressed"]
input_vec = token.texts_to_sequences(input_vec)
input_vec = sequence.pad_sequences(input_vec, maxlen=max_seq_len)
label_prob = model.predict(input_vec)
label_pred = label_prob.argmax(axis=-1)
print(label_encoder.inverse_transform(label_pred))

['pos' 'neg' 'pos' 'neg' 'neg']


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Comparision of CNN's and RNN's results

The same polarity dataset was trained and tested over both CNNs and RNNs where we found the accuracy of RNNs using LSTMs is around 92% which is more than what we obtained using CNNs i.e 88%. We can also see that the first predicted sentence being explicitly stated good is not considered positive while LSTMs performed better over the same.

References:

https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html
